<a href="https://colab.research.google.com/github/piyush-an/INFO7374_Predict_StockPrice/blob/main/notebooks/2a_FeatureMart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash

pip install pandas yfinance ta seaborn matplotlib pandas-datareader jinja2 fredapi openpyxl xgboost scikit-learn statsmodels mlflow

if [ ! -f "ads_index_most_current_vintage.xlsx" ]; then
    wget https://www.philadelphiafed.org/-/media/frbp/assets/surveys-and-data/ads/ads_index_most_current_vintage.xlsx
fi
if [ ! -f "F-F_Research_Data_Factors_daily.CSV" ]; then
  wget https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip
  wget https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_5_Factors_2x3_daily_CSV.zip
	unzip F-F_Research_Data_Factors_daily_CSV.zip
  unzip F-F_Research_Data_5_Factors_2x3_daily_CSV.zip
fi

In [3]:
# Importing ALl Libraries
import pandas as pd
from fredapi import Fred

In [14]:
# Step1: Getting the FamaFrench 5 factor data
data_ff5 = pd.read_csv("./F-F_Research_Data_5_Factors_2x3_daily.CSV", skiprows=3)
data_ff5.head()

,Unnamed: 0,Mkt-RF,SMB,HML,RMW,CMA,RF
0,19630701,-0.67,0.02,-0.35,0.03,0.13,0.012
1,19630702,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,19630703,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,19630705,0.40,0.09,-0.28,0.07,-0.30,0.012
4,19630708,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [15]:
data_ff5.rename(columns={"Unnamed: 0": "date"}, inplace=True)
data_ff5["date"] = pd.to_datetime(data_ff5["date"], format='%Y%m%d')

In [16]:
data_ff5.head()

,date,Mkt-RF,SMB,HML,RMW,CMA,RF
0,1963-07-01,-0.67,0.02,-0.35,0.03,0.13,0.012
1,1963-07-02,0.79,-0.28,0.28,-0.08,-0.21,0.012
2,1963-07-03,0.63,-0.18,-0.10,0.13,-0.25,0.012
3,1963-07-05,0.40,0.09,-0.28,0.07,-0.30,0.012
4,1963-07-08,-0.63,0.07,-0.20,-0.27,0.06,0.012


In [17]:
# Making the final famafrench dataset with the the date index
df_ff5 = data_ff5.set_index("date")

In [18]:
# Step2: Getting the data of the ADS index
data_ADS = pd.read_excel("./ads_index_most_current_vintage.xlsx")
data_ADS.head()

,Unnamed: 0,ADS_Index
0,1960:03:01,-0.572930
1,1960:03:02,-0.621487
2,1960:03:03,-0.666982
3,1960:03:04,-0.709432
4,1960:03:05,-0.748855


In [19]:
data_ADS.rename(columns={"Unnamed: 0": "date"}, inplace=True)
data_ADS["date"] = pd.to_datetime(data_ADS["date"], format="%Y:%m:%d")
data_ADS.head()

,date,ADS_Index
0,1960-03-01,-0.572930
1,1960-03-02,-0.621487
2,1960-03-03,-0.666982
3,1960-03-04,-0.709432
4,1960-03-05,-0.748855


In [20]:
# Making the final ADS index data with date as the index
df_ADS = data_ADS.set_index("date")

In [21]:
# Step3: Getting the data using Fred API
fred = Fred(api_key="dfd5e2b3a7c9a714430cdf450699e698")

varList = [
    "T10Y3M",
    "OBMMIJUMBO30YF",  # term premium 10yr-3mon, 30 yr mortgage
    "DEXUSEU",
    "DEXJPUS",
    "DEXUSUK",  # spot exchange rates to EUR, JPY, GBP
    "CBBTCUSD",
    "CBETHUSD",  # cryptocurrencies
    "T10YIE",
    "DCOILBRENTEU",  # breakeven inflation + brent oil price
    "VIXCLS",  # implied volatilities
    "DAAA",
    "DBAA",  # corporate bond yield
    "NIKKEI225",
    "AMERIBOR",
    "T10YIE",
    "T5YIE",
    "BAMLH0A0HYM2",
    "BAMLH0A0HYM2EY",
    "DGS10",
    "DGS1",
    "RIFSPPFAAD90NB",
    "DCPN3M",
    "DCPF1M",
    "DCOILWTICO",
    "DHHNGSP",
    "USRECD",
    "USRECDM",
    "USRECDP",
]

SP500 = fred.get_series("SP500")
SP500.name = "SP500"
df_fred = SP500
# merge data series
for i in range(0, len(varList)):
    data = fred.get_series(varList[i])
    data.name = varList[i]
    df_fred = pd.merge(df_fred, data, left_index=True, right_index=True)

In [ ]:
df_fred.head()

,SP500,T10Y3M,OBMMIJUMBO30YF,DEXUSEU,DEXJPUS,DEXUSUK,CBBTCUSD,CBETHUSD,T10YIE_x,DCOILBRENTEU,...,DGS10,DGS1,RIFSPPFAAD90NB,DCPN3M,DCPF1M,DCOILWTICO,DHHNGSP,USRECD,USRECDM,USRECDP
2017-01-03,2257.83,1.92,4.432,1.0416,117.68,1.2256,1020.67,9.59,1.98,55.05,...,2.45,0.89,0.99,0.77,0.70,52.36,3.41,0.0,0.0,0.0
2017-01-04,2270.75,1.93,4.355,1.0476,117.38,1.2299,1130.30,11.17,1.99,54.57,...,2.46,0.87,0.95,0.79,0.71,53.26,3.42,0.0,0.0,0.0
2017-01-05,2269.00,1.85,4.377,1.0598,115.46,1.2415,1007.00,10.23,1.95,54.99,...,2.37,0.83,0.91,0.78,0.65,53.77,3.42,0.0,0.0,0.0
2017-01-06,2276.98,1.89,4.385,1.0560,116.85,1.2314,895.71,10.14,1.96,55.90,...,2.42,0.85,0.90,0.71,0.73,53.98,3.38,0.0,0.0,0.0
2017-01-09,2268.90,1.88,4.302,1.0576,116.07,1.2167,902.66,10.39,1.95,54.39,...,2.38,0.82,0.90,0.76,0.74,51.95,3.14,0.0,0.0,0.0


In [22]:
# Training Period 2018-2023(august)

start_date = "2018-01-01"
end_date = "2023-12-31"

# Keeping only the data that fits into the training period.
mask = (df_ff5.index >= start_date) & (df_ff5.index <= end_date)
df_ff5 = df_ff5.loc[mask]

mask = (df_ADS.index >= start_date) & (df_ADS.index <= end_date)
df_ADS = df_ADS.loc[mask]

DATA_part1 = pd.merge(df_ff5, df_ADS, left_index=True, right_index=True)
DATA = pd.merge(DATA_part1, df_fred, left_index=True, right_index=True)
DATA.reset_index(inplace=True)
DATA = DATA.rename(columns={"index": "Date"})

In [23]:
DATA.head()

,Date,Mkt-RF,SMB,HML,RMW,CMA,RF,ADS_Index,SP500,T10Y3M,...,DGS10,DGS1,RIFSPPFAAD90NB,DCPN3M,DCPF1M,DCOILWTICO,DHHNGSP,USRECD,USRECDM,USRECDP
0,2018-01-02,0.85,0.30,-0.22,-0.50,0.31,0.006,-0.283936,2695.81,1.02,...,2.46,1.83,1.68,1.58,1.52,60.37,6.24,0.0,0.0,0.0
1,2018-01-03,0.59,-0.46,-0.18,-0.70,-0.06,0.006,-0.304869,2713.06,1.03,...,2.44,1.81,1.62,1.57,1.44,61.61,6.24,0.0,0.0,0.0
2,2018-01-04,0.42,-0.22,0.24,-0.02,0.31,0.006,-0.323642,2723.99,1.05,...,2.46,1.82,1.65,1.59,1.47,61.98,4.65,0.0,0.0,0.0
3,2018-01-05,0.66,-0.37,-0.26,0.37,-0.38,0.006,-0.340263,2743.15,1.08,...,2.47,1.80,1.56,1.58,1.47,61.49,NaN,0.0,0.0,0.0
4,2018-01-08,0.19,-0.19,0.05,-0.07,0.01,0.006,-0.378450,2747.71,1.04,...,2.49,1.79,1.61,1.57,1.47,61.73,2.89,0.0,0.0,0.0


In [24]:
DATA.shape

(1468, 37)

In [25]:
filename = f"NVDA_feature_mart_b.csv"
DATA.to_csv(filename, index=True)